## Homework Grading Tools

Tools to help download and grade homeworks.

In [1]:
import requests
import time
import os, os.path

Authentication tokens. **WARNING: KEEP THESE PRIVATE**!!!

In [2]:
host = 'https://growth.dirac.institute'
api_url = f'{host}/hub/api'
token = 'dde9a43855154edc9f4df60103e8912c'

Function definitions:

In [4]:
def start_server(name):
    # Start a user's server
    #
    # returns the URL to server's API if running, None otherwise
    # https://jupyterhub.readthedocs.io/en/stable/_static/rest-api/index.html#operation--users--name--server-post
    r = requests.post(api_url + f'/users/{name}/server', headers={'Authorization': 'token %s' % token})
    if r.status_code == 400:
        # verify the server is running
        r = requests.get(api_url + f'/users/{name}', headers={'Authorization': 'token %s' % token})
        r.raise_for_status()
        ret = r.json()
        assert ret['pending'] is None

        # return the API url
        return f"{host}/{ret['server']}api"

    r.raise_for_status()

def list_all_users():
    r = requests.get(api_url + '/users', headers={ 'Authorization': 'token %s' % token })
    r.raise_for_status()

    return [ x['name'] for x in r.json() ]

def start_all_servers():
    #
    # Start all users' Jupyter servers
    #
    r = requests.get(api_url + '/users', headers={ 'Authorization': 'token %s' % token })
    r.raise_for_status()

    servers = {}
    waitstart = False
    for x in r.json():
        name, server, pending = x['name'], x['server'], x['pending']
        if server is None:
            waitstart = True
            if pending is None:
                print(f"❌ {name} ... starting")
                start_server(name)
            else:
                print(f"⏳ {name}")
        else:
            servers[name] = f"{host}/{server}api/"
            #print(f"✅ {name}")
    return waitstart, servers

def start_all_servers_loop():
    from IPython.display import clear_output
    while True:
        clear_output(wait=True)
        pending, servers = start_all_servers()
        if not pending:
            break
        time.sleep(20)
        
    clear_output(wait=True)
    print("Done! Ready to download.")
    return servers

def download_file(api, path, tag=None, destdir=None, clobber=False):
    #
    # Download a file from a user's Jupyter server
    #
    import base64, os.path

    # found this documented only in the source code at
    # https://github.com/jupyter/notebook/blob/b8b66332e2023e83d2ee04f83d8814f567e01a4e/notebook/static/services/contents.js#L77
    params = {'type': 'file', 'format': 'base64'}
    r = requests.get(f"{api}contents/{path}", headers={'Authorization': 'token %s' % token },
                    params=params)
    r.raise_for_status()
    ret = r.json()
    content = base64.b64decode(ret['content'])

    _, outfn = os.path.split(path)
    if tag:
        base, ext = os.path.splitext(outfn)
        outfn = f"{base}.{tag}{ext}"

    if clobber is False and os.path.exists(outfn):
        raise FileExistsError(outfn)

    if destdir is not None:
        outfn = os.path.join(destdir, outfn)
        
    print(f"writing to {outfn}")
    with open(outfn, "wb") as fp:
        fp.write(content)

def download_homeworks(servers, hw='HW1'):
    # Download all homeworks for all usersbb
    import time
    tm = time.strftime('%Y%m%d')

    # make subdirectories to save toam
    if not os.path.exists(hw):
        os.makedirs(hw)
    
    for name, api in servers.items():
        tag = f"{tm}.{name}"
        path = f'astr-324-s20-homeworks/astr324-s20-{hw}.ipynb'
        download_file(api, path, tag=tag, destdir=hw, clobber=True)

#servers = start_all_servers()

#api = start_server('mjuric')
#api

# download_file(servers['mjuric'], 'astr-324-s20-homeworks/astr324-s20-HW1.ipynb', tag='mjuric', clobber=True)

#servers = { 'mjuric': servers['mjuric']}
#download_homeworks(servers, hw='HW1')

In [5]:
list_all_users()

['igorandreoni',
 'stevenstetzler',
 'mjuric',
 'dlakaplan',
 'davidkaplantest',
 'ddobie',
 'mansikasliwal',
 'rremigio',
 'ms1228',
 'vedshenoy',
 'knights-templars',
 'arvindb95',
 'rmquimby',
 'dperley',
 'tahumada',
 'pradipgatkine',
 'ejh-ljmu',
 'blancmatter',
 'charlotteaward',
 'spamfour',
 'viraj21197',
 'yashvi-sharma',
 'yaoyuhan',
 'cfremling',
 'harshkumar13',
 'shreyasahasram08',
 'dekishalay',
 'chris5281',
 'asaguescar',
 'simeonreusch',
 'avondale17',
 'hjsreehari',
 'anke-astro',
 'parulj3795',
 'varun2501',
 'tkillestein',
 'temuller',
 'kiranjayasurya',
 'ry-c123',
 'sandeep-rout',
 'swayamtrupta',
 'kruthi24',
 'gravitas21',
 'klukosiute',
 'joemichail',
 'schaftler',
 'derin-wilson',
 'wheresmysamwich',
 'sps-vic',
 'prusinski',
 'danieleuts',
 'jennyramos',
 'krrawlins',
 'tkolcu',
 'kaustavkdas',
 'mattleung10',
 'ff-dirirsa',
 'kaeleeparker',
 'paulomipaul',
 'uwastrohendrik',
 'patrickaleo',
 'larissaamaral',
 'elismar2000',
 'nanditakhetan',
 'snehamudambi',

In [16]:
servers = start_all_servers_loop()

Done! Ready to download.


In [17]:
download_homeworks(servers, hw='HW1')

writing to HW1/astr324-s20-HW1.20200417.landc2.ipynb
writing to HW1/astr324-s20-HW1.20200417.mjuric.ipynb
writing to HW1/astr324-s20-HW1.20200417.aberres.ipynb
writing to HW1/astr324-s20-HW1.20200417.maria8ch.ipynb
writing to HW1/astr324-s20-HW1.20200417.liyanz.ipynb
writing to HW1/astr324-s20-HW1.20200417.libraj.ipynb
writing to HW1/astr324-s20-HW1.20200417.mlindner.ipynb
writing to HW1/astr324-s20-HW1.20200417.wmmunhin.ipynb
writing to HW1/astr324-s20-HW1.20200417.davidfro.ipynb
writing to HW1/astr324-s20-HW1.20200417.danlozan.ipynb
writing to HW1/astr324-s20-HW1.20200417.thomak22.ipynb
writing to HW1/astr324-s20-HW1.20200417.lexie98.ipynb
writing to HW1/astr324-s20-HW1.20200417.msthomps.ipynb
writing to HW1/astr324-s20-HW1.20200417.cayenne.ipynb
writing to HW1/astr324-s20-HW1.20200417.bergea4.ipynb
writing to HW1/astr324-s20-HW1.20200417.tw94.ipynb
writing to HW1/astr324-s20-HW1.20200417.ginder.ipynb
writing to HW1/astr324-s20-HW1.20200417.nmarosan.ipynb
writing to HW1/astr324-s20-H